In [1]:
import os
import re
import pandas as pd
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import logging
from tqdm import tqdm
import torch

c:\Users\durud\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def parse_hotel_reviews(data_dir):
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    # Updated regex pattern to handle flexible spacing and separators
    date_pattern = re.compile(r'(\b(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{1,2}\s\d{4})\s+([^\t\n]*)\s+(.*)')
    data_frames = []

    for root, dirs, files in os.walk(data_dir):
        city = os.path.basename(root)
        for file in files:
            file_path = os.path.join(root, file)
            hotel_name = os.path.splitext(file)[0]  # Assuming the file has no extension
            try:
                with open(file_path, 'r', encoding='ISO-8859-1') as f:
                    content = f.read()

                if not content:
                    logging.warning(f'File {file_path} is empty.')
                    continue

                reviews = date_pattern.findall(content)
                if not reviews:
                    logging.warning(f'No reviews found in file {file_path}. Content: {content[:500]}')
                    continue

                reviews_data = [{'hotel_name': hotel_name, 'hotel_city': city, 'review_date': date, 'hotel_review': review.strip()} for date, _, title, review in reviews]
                if reviews_data:
                    data_frames.append(pd.DataFrame(reviews_data))

            except Exception as e:
                logging.error(f'Error processing file {file_path}: {e}')
                continue

    if data_frames:
        review_df = pd.concat(data_frames, ignore_index=True)
    else:
        review_df = pd.DataFrame(columns=['hotel_name', 'hotel_city', 'review_date', 'hotel_review'])

    return review_df

dataset = 'beijing_deberta_microsoft'
data_dir = 'data'
review_df = parse_hotel_reviews(data_dir)
print(review_df.head())
review_df.to_csv(f'csv/{dataset}_derlenmis_reviews.csv', index=False)

                            hotel_name hotel_city  review_date  \
0  china_beijing_aloft_beijing_haidian    beijing  Oct 12 2009   
1  china_beijing_aloft_beijing_haidian    beijing  Sep 25 2009   
2  china_beijing_aloft_beijing_haidian    beijing   Aug 4 2009   
3  china_beijing_aloft_beijing_haidian    beijing  Jul 17 2009   
4  china_beijing_aloft_beijing_haidian    beijing  May 30 2009   

                                        hotel_review  
0  I stayed in this hotel for one night. As this ...  
1  Stayed two nights at Aloft on the most recent ...  
2  We stayed at the Aloft Beijing Haidian for 5 n...  
3  I am glad to be the first person to post photo...  
4  Stayed there for one night. The hotel is locat...  


In [4]:
review_df = pd.read_csv(f'csv/{dataset}_derlenmis_reviews.csv')
review_df = review_df.dropna(subset=['hotel_review'])
review_df['processed_review'] = review_df['hotel_review'].str.lower().str.replace('\t', ' ', regex=False)
review_df = review_df.drop('hotel_review', axis=1)
review_df = review_df.drop('review_date', axis=1)
review_df.to_csv(f'csv/{dataset}_processed_reviews.csv', index=False)

In [5]:
deberta_tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")
deberta_model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-base")
deberta_pipeline = pipeline("sentiment-analysis", model=deberta_model, tokenizer=deberta_tokenizer)

c:\Users\durud\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\durud\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\durud\.cache\huggingface\hub\models--microsoft--deberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate 

In [6]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

aspects = ['cleanliness', 'room', 'service', 'location', 'value', 'safety', 'comfort', 'transportation', 'noise']
deberta_weights = {'NEGATIVE': -1, 'NEUTRAL': 0, 'POSITIVE': 1}

aspect_keywords = {
    'cleanliness': ['clean', 'dirty', 'smell', 'stink', 'stunk', 'filthy'],
    'room': ['room', 'bed', 'suite', 'large'],
    'service': ['service', 'staff', 'help', 'support'],
    'location': ['location', 'close', 'area', 'far'],
    'value': ['value', 'worth', 'price'],
    'safety': ['safe', 'safety', 'secure', 'danger', 'dangerous'],
    'comfort': ['comfort', 'comfortable', 'uncomfortable'],
    'transportation': ['bus', 'metro', 'station', 'close', 'walk'],
    'noise': ['sound', 'volume', 'noisy', 'noise']
}
review_df = pd.read_csv(f'csv/{dataset}_processed_reviews.csv')

batch_size = 16
def process_batch(batch_reviews, aspect):
    for aspect in aspects:
        review_df[f'{aspect}_score'] = 0
        aspect_reviews = [f"{aspect}: {review}" for review in batch_reviews]
        inputs = absa_tokenizer(aspect_reviews, return_tensors="pt", padding=True, truncation=True)
        outputs = absa_model(**inputs)
        probs = F.softmax(outputs.logits, dim=-1)
        return probs.detach().numpy()

def is_aspect_mentioned(review, aspect):
    keywords = aspect_keywords[aspect]
    return any(keyword in review for keyword in keywords)



review_df = pd.read_csv(f'csv/{dataset}_processed_reviews.csv')

IndentationError: expected an indented block after 'for' statement on line 21 (4186952136.py, line 22)

In [15]:
for aspect in aspects:
    logging.info(f"Starting processing for aspect: {aspect}")
    for i in tqdm(range(0, len(review_df), batch_size), desc=f"Batches for {aspect}"):
        batch_reviews = review_df['processed_review'][i:i + batch_size]
        batch_index = batch_reviews.index
        aspect_mentioned = [is_aspect_mentioned(review, aspect) for review in batch_reviews]
        if any(aspect_mentioned):
            filtered_reviews = [review for review, mentioned in zip(batch_reviews, aspect_mentioned) if mentioned]
            probs = process_batch(filtered_reviews, aspect)
            scores = [sum(weights[sentiment] * probs[j, k] for k, sentiment in enumerate(['negative', 'neutral', 'positive'])) for j in range(len(probs))]
            score_index = [index for index, mentioned in zip(batch_index, aspect_mentioned) if mentioned]
            review_df.loc[score_index, f'{aspect}_score'] = scores
        else:
            review_df.loc[batch_index, f'{aspect}_score'] = 0

    logging.info(f"Finished processing for aspect: {aspect}")

2024-05-16 02:14:36,919 - INFO - Starting processing for aspect: cleanliness
Batches for cleanliness:   0%|          | 0/3 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
C:\Users\durud\AppData\Local\Temp\ipykernel_9108\2707505802.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.9822808085009456, 0.8573983013629913, 0.4965289235115051, 0.8957091234624386, 0.9326148964464664, 0.8547404706478119]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  review_df.loc[score_index, f'{aspect}_score'] = scores
Batches for cleanliness: 100%|██████████| 3/3 [00:22<00:00,  7.58s/it]
2024-05-16 02:14:59,655 - INFO - Finished processing for aspect: cleanliness
2024-05-16 02:14:59,656 - INFO - Starting processing for aspect: room
Batches for room:   0%|          | 0/3 [

In [16]:
# Ensure aspect score columns are numeric
for aspect in aspects:
    review_df[f'{aspect}_score'] = pd.to_numeric(review_df[f'{aspect}_score'], errors='coerce')

# Aggregate scores by hotel and city
aggregated_scores = review_df.groupby(['hotel_name', 'hotel_city']).agg({
    'cleanliness_score': 'mean',
    'room_score': 'mean',
    'service_score': 'mean',
    'location_score': 'mean',
    'value_score': 'mean',
    'safety_score': 'mean',
    'comfort_score': 'mean',
    'transportation_score': 'mean',
    'noise_score': 'mean'
}).reset_index()

# Save the aggregated scores
aggregated_scores.to_csv(f'csv/{dataset}_aggregated_hotel_scores.csv', index=False)


In [17]:
display(aggregated_scores)

,hotel_name,hotel_city,cleanliness_score,room_score,service_score,location_score,value_score,safety_score,comfort_score,transportation_score,noise_score
0,china_beijing_aloft_beijing_haidian,beijing,0.538653,0.697316,0.563874,0.401005,0.466420,0.000000,0.314276,0.322668,0.000000
1,china_beijing_ascott_beijing,beijing,0.313631,0.576998,0.474162,0.502369,0.107623,0.033085,0.193290,0.450315,0.031618
2,china_beijing_autumn_garden_courtyard_hotel,beijing,0.156190,0.463816,0.661807,0.489531,0.158361,0.000000,0.321703,0.774139,0.000000
